## LangChain
+ Memory
+ Chains
+ RAG
+ Agent
+ Evaluation

## Use LangChain build an app to answer based your own data
+ Document load
+ Document split
+ VectorDB
+ Retrieval 


## Preparation

```python
pip install langchain
pip install langchain-openai, langchain-community
ollama pull llama2
```

In [1]:
from langchain_openai import ChatOpenAI
from tool.openai import get_openai_key

llm = ChatOpenAI(openai_api_key=get_openai_key())

In [2]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x15b04da50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x15b638f50>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [3]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print("提示模版中的第一个提示:", prompt_template.messages[0].prompt)

customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)
print("用提示模版中生成的第一条客户消息:", customer_messages[0])


提示模版中的第一个提示: input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
用提示模版中生成的第一条客户消息: content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [4]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print("提示模版中的第一条客户消息内容:", service_messages[0].content)
service_response = llm(service_messages) 
print("模型得到的回复邮件:", service_response.content)

提示模版中的第一条客户消息内容: Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


模型得到的回复邮件: Ahoy there, valued customer! 'Tis with regret that I must inform ye that the warranty be not coverin' the cost of cleanin' yer kitchen, as 'tis yer own fault for misusin' yer blender by forgettin' to put the lid on afore startin' it. 'Tis a tough break, me hearties! Fare thee well!


In [7]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
review_template = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
Format the output as JSON with the following keys:
gift
delivery_days
price_value
text: {text}
"""
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(review_template)
chat = prompt | llm
response = chat.invoke(input={"text": customer_review}) 
print("回复内容:",response.content)

回复内容: {
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [8]:
review_template_2 = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
text: {text}
{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")
response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [9]:
messages = prompt.format_messages(text=customer_review,
format_instructions=format_instructions)
print("提示消息:", messages[0].content)

提示消息: For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.
text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "``

In [15]:
chat = prompt | llm | output_parser

input = {
    "text": customer_review,
    "format_instructions": format_instructions
}

response = chat.invoke(input=input)
print(response)

{'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [19]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "what's up"})

In [40]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

In [41]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory,
)

In [42]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?"

In [43]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?"

In [44]:
conversation.invoke(input="I want to talk about large language models.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?
Human: I want to talk about large language models.
AI:

> Finished chain.


{'input': 'I want to talk about large language models.',
 'history': "Human: Hi there!\nAI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?\nHuman: I'm doing well! Just having a conversation with an AI.\nAI:  That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?",
 'response': ' Sure, I am very familiar with large language models. They are a type of AI model that uses a large amount of data to generate human-like text. They have been used for various tasks such as language translation, text summarization, and even creating entire articles. Some popular examples of large language models include GPT-3, BERT, and Transformer. Is there anything else you would like to know about them?'}

In [45]:
print(memory.load_memory_variables({}))

{'history': 'System: \nThe human greets the AI and the AI introduces itself as an AI created by OpenAI, constantly learning and improving through machine learning. The human mentions having a conversation with an AI and the AI expresses its interest in engaging in conversations and learning more about human interactions. The AI asks if there is anything specific the human would like to talk about.\nHuman: I want to talk about large language models.\nAI:  Sure, I am very familiar with large language models. They are a type of AI model that uses a large amount of data to generate human-like text. They have been used for various tasks such as language translation, text summarization, and even creating entire articles. Some popular examples of large language models include GPT-3, BERT, and Transformer. Is there anything else you would like to know about them?'}


SimpleSequentialChains: 1 input, 1 output
As follows, the second chain's input (company_name) is the first chain's output.

In [47]:
from langchain.chains import SimpleSequentialChain, LLMChain

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)

product = "Queen Size Sheet Set"
overall_simple_chain.invoke(product)




> Entering new SimpleSequentialChain chain...


"Royal Rest Linens"


"Experience the ultimate luxury in bedding with Royal Rest Linens - where comfort and elegance meet for a regal night's sleep."

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': '\n\n"Experience the ultimate luxury in bedding with Royal Rest Linens - where comfort and elegance meet for a regal night\'s sleep."'}

SequentialChain: multi input, multi output


In [49]:
from langchain.chains import SequentialChain

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

review = """Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre.
J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou
contrefaçon !?"""
response = overall_chain.invoke(review)
print(response)



> Entering new SequentialChain chain...

> Finished chain.
{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre.\nJ'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou\ncontrefaçon !?", 'English_Review': "\n\nI find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?", 'summary': '\n\nThe reviewer is disappointed with the taste and foam of the product, and suspects it may be an old or counterfeit batch.', 'followup_message': "\n\nRéponse: Je suis désolé que vous ayez été déçu par le goût et la mousse du produit. Nous prenons la qualité de nos produits très au sérieux et nous enquêterons sur cette question. Il est possible qu'il s'agisse d'un lot ancien ou contrefait et nous ferons tout notre possible pour résoudre ce problème. Merci de nous avoir informés de votre expérience et nous espérons que vous nous donnerez une autre chance à l'avenir

Route Chain


In [50]:
from langchain.chains.router import MultiPromptChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.
Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.
Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.
Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.
Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    }
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.
<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.
<< CANDIDATE PROMPTS >>
{destinations}
<< INPUT >>
{{input}}
<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [51]:
print(chain.invoke("What is black body radiation?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
{'input': 'What is black body radiation?', 'text': '\n\nBlack body radiation is the thermal electromagnetic radiation emitted by an object due to its temperature. It is called "black body" because it absorbs all radiation that falls on it and emits radiation at all wavelengths. This type of radiation follows a specific distribution known as the Planck\'s law, which describes the intensity of radiation at different wavelengths based on the temperature of the object. Black body radiation is an important concept in understanding the behavior of objects at high temperatures, such as stars and planets.'}


In [52]:
print(chain.invoke("what is 2 + 2"))



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.
{'input': 'what is 2 + 2', 'text': '\n\nAI: Thank you for the compliment! As a language AI, I am not able to solve math problems like a human mathematician. However, I can provide the answer to your question. 2 + 2 is equal to 4.'}


In [53]:
print(chain.invoke("Why does every cell in our body contain DNA?"))



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.
{'input': 'Why does every cell in our body contain DNA?', 'text': "\n\nEvery cell in our body contains DNA because it is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the genetic code that determines our physical characteristics, such as eye color, hair color, and height, as well as our susceptibility to certain diseases. It is essential for the growth, repair, and maintenance of our body's cells and tissues. Without DNA, our cells would not be able to function properly and our bodies would not be able to survive."}


In [54]:
print(chain.invoke("Can you give me some introduction about ancient China? Such as Qing Dynasty."))



> Entering new MultiPromptChain chain...
History: {'input': 'Can you give me some introduction about ancient China? Such as Qing Dynasty.'}
> Finished chain.
{'input': 'Can you give me some introduction about ancient China? Such as Qing Dynasty.', 'text': '\n\nSure, the Qing Dynasty was the last imperial dynasty of China, ruling from 1644 to 1912. It was founded by the Manchu people, who conquered the Ming Dynasty and established their own rule. The Qing Dynasty was known for its expansion and consolidation of territories, including Taiwan, Mongolia, and Tibet. It also saw significant cultural and economic developments, such as the growth of trade and the introduction of new crops like potatoes and corn. However, the Qing Dynasty also faced challenges, such as rebellions and conflicts with foreign powers, leading to its eventual downfall and the establishment of the Republic of China.'}


Answer Question Based on Documents

In [56]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from IPython.display import display, Markdown
import pandas as pd

file = './data/OutdoorClothingCatalog_1000.csv'

In [57]:
loader = CSVLoader(file_path=file)
data = pd.read_csv(file,usecols=[1, 2]) 
data.head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


In [58]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."
response = index.query(query)
display(Markdown(response))

/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')




| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, wicks moisture, abrasion resistant | SPF 50+, blocks 98% of harmful UV rays |

In [59]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embed = embeddings.embed_query("Hi my name is Harrison")
print("\n\033[32m向量表征的长度: \033[0m \n", len(embed))
print("\n\033[32m向量表征前5个元素: \033[0m \n", embed[:5])


向量表征的长度:  
 1536

向量表征前5个元素:  
 [-0.021993802851855686, 0.006747527976699374, -0.018252847709138518, -0.03916704653175714, -0.013997197145759574]


In [61]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [62]:
query = "Please suggest a shirt with sunblocking"
docs = vector.similarity_search(query)
print("\n\033[32m返回文档的个数: \033[0m \n", len(docs)) 
print("\n\033[32m第一个文档: \033[0m \n", docs[0])


返回文档的个数:  
 4

第一个文档:  
 page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.' metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 255}


In [64]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [67]:
display(Markdown(response))



| Name | Description | Fabric & Care | Additional Features |
| --- | --- | --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ protection | 78% nylon, 22% Lycra Xtra Life fiber | Slightly Fitted, handwash, line dry |
| SunSmart™ Sleeveless Button-Up Shirt | Five-star sleeveless button-up shirt with UPF 50+ protection | Shell: 71% nylon, 29% polyester; Cape lining: 100% polyester | Slightly Fitted, machine wash and dry, wrinkle resistant, low-profile pockets |
| Ultracomfortable Sun Protection Shirt | Lightest hot-weather shirt with UPF 50+ protection | 52% polyester, 48% nylon | Machine wash and dry, wrinkle-free, quick-drying, front and back venting |
| Men's Sun-Protection Shirt | Lightweight shirt with UPF 50+ protection | Shell: 71% nylon, 29% polyester; Lining: 100% polyester knit mesh | Traditional Fit, machine wash and dry, wrinkle resistant, front and back cape venting |

Chain Type
+ stuff: 将所有查询得到的文档组合成一个文档传入下一步
+ Map Reduce: 将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行
+ Refine: 用于循环许多文档，际上是迭代的，建立在先前文档的答案之上，非常适合前后因果 信息并随时间逐步构建答案
+ Map Re-rank: 对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依 赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分


In [74]:
retriever = vector.as_retriever()

qa_stuff = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever, verbose=True)
query = "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."
response = qa_stuff.invoke(query)
display(Markdown(response['result']))



> Entering new RetrievalQA chain...

> Finished chain.


 
| Shirt | Sun Protection | Summary |
| --- | --- | --- |
| Five-star sleeveless button-up shirt | SunSmart™ UPF 50+ | Softly shapes the body, falls at hip, wrinkle resistant, low-profile pockets and side shaping |
| Men’s sun-protection shirt | UPF 50+ | Lightweight, relaxed fit, wrinkle resistant, front and back cape venting |
| Sun Shield Shirt | UPF 50+ | Lightest hot-weather shirt, relaxed fit, wrinkle resistant, front and back cape venting, two front bellows pockets |

In [72]:
display(Markdown(response['result']))



| Name | Description | Size & Fit | Fabric & Care | Additional Features |
| --- | --- | --- | --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ protection. | Slightly Fitted, Falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber. Handwash, line dry. | Guaranteed to protect from harmful UV rays. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ protection. | Traditional Fit, Relaxed through chest, sleeve, and waist. | 100% polyester. Wrinkle-resistant. | Front and back cape venting, two front bellows pockets. |